# Connecting to the database

In [41]:
import pandas as pd

In [1]:
from db_tools.setup import setup
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

# Getting a SparkSession

In [2]:
spark = setup()

23/10/28 19:42:16 WARN Utils: Your hostname, luan-Dell-G15-5520 resolves to a loopback address: 127.0.1.1; using 192.168.1.12 instead (on interface wlp0s20f3)
23/10/28 19:42:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/10/28 19:42:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


SparkSession available as "spark"


In [3]:
def show_query(query,SparkSession = spark,n = 20):
    return SparkSession.sql(query).show()

# Base query - basic average

In [4]:
query = """
		SELECT
			AVG (rental_rate)
		FROM
			film
"""
show_query(query)

+----------------+
|avg(rental_rate)|
+----------------+
|        2.980000|
+----------------+



# Subquery example

In [5]:
query = """
SELECT
	film_id,
	title,
	rental_rate
FROM
	film
WHERE
	rental_rate > (
		SELECT
			AVG (rental_rate)
		FROM
			film
	);
"""
show_query(query)

+-------+-------------------+-----------+
|film_id|              title|rental_rate|
+-------+-------------------+-----------+
|    133|    Chamber Italian|       4.99|
|    384|   Grosse Wonderful|       4.99|
|      8|    Airport Pollock|       4.99|
|     98|  Bright Encounters|       4.99|
|      2|     Ace Goldfinger|       4.99|
|      3|   Adaptation Holes|       2.99|
|      4|   Affair Prejudice|       2.99|
|      5|        African Egg|       2.99|
|      6|       Agent Truman|       2.99|
|      7|    Airplane Sierra|       4.99|
|      9|      Alabama Devil|       2.99|
|     10|   Aladdin Calendar|       4.99|
|     13|        Ali Forever|       4.99|
|     15|       Alien Center|       2.99|
|     16|    Alley Evolution|       2.99|
|     20|Amelie Hellfighters|       4.99|
|     21|    American Circus|       4.99|
|     22|  Amistad Midsummer|       2.99|
|     24|   Analyze Hoosiers|       2.99|
|     25|        Angels Life|       2.99|
+-------+-------------------+-----

# Checking movies returned between '2005-05-29' and '2005-05-30'

In [6]:
query = """SELECT
	film_id,
	title
FROM
	film
WHERE
	film_id IN (
		SELECT
			inventory.film_id
		FROM
			rental
		INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
		WHERE
			return_date BETWEEN '2005-05-29'
		AND '2005-05-30'
	);
 """
show_query(query)

+-------+--------------------+
|film_id|               title|
+-------+--------------------+
|    471|     Island Exorcist|
|    858|       Submarine Bed|
|    255|      Driving Polish|
|    898|     Tourist Pelican|
|    772|          Sea Virgin|
|    300|       Falcon Volume|
|    914|        Trouble Date|
|    103|  Bucket Brotherhood|
|    973|           Wife Turn|
|    388|       Gunfight Moon|
|    875|   Talented Homicide|
|    233|     Disciple Mother|
|    694|  Prejudice Oleander|
|     52|Ballroom Mockingbird|
|    715|    Range Moonwalker|
|    348|         Gandhi Kwai|
|    164|       Coast Rainbow|
|    139|       Chasing Fight|
|    295| Expendable Stallion|
|    868|       Superfly Trip|
+-------+--------------------+
only showing top 20 rows



# EXISTS clause: checking clients who paid

In [7]:
query = """
SELECT
	first_name,
	last_name
FROM
	customer
WHERE
	EXISTS (
		SELECT
			1
		FROM
			payment
		WHERE
			payment.customer_id = customer.customer_id
	);

"""
show_query(query)

+----------+---------+
|first_name|last_name|
+----------+---------+
|   Eleanor|     Hunt|
|   Darrell|    Power|
|      Dean|    Sauer|
|     Tyler|     Wren|
|     Lydia|    Burke|
|      Sean| Douglass|
|    Tyrone|    Asher|
|    Brenda|   Wright|
|     Elmer|      Noe|
|      Anne|   Powell|
|    Rhonda|  Kennedy|
|    Vickie|   Brewer|
|       Jim|      Rea|
|      Ross|     Grey|
|      Rose|   Howard|
|     Lloyd|     Dowd|
|   Heather|   Morris|
|      Irma|  Pearson|
|    Herman|   Devore|
|    Marion|   Ocampo|
+----------+---------+
only showing top 20 rows



# ANY clause

## Spark documentation - is different from SQL queries

In [105]:
query = "SELECT ANY(col) FROM VALUES (true), (false), (false) AS tab(col);"
show_query(query)

+--------+
|any(col)|
+--------+
|    true|
+--------+



## Basic query - find the maximum duration for each genre

In [8]:
base_query = """
SELECT 
    category.name,
    MAX( length ) AS maximum_duration
FROM
    film
INNER JOIN film_category
        USING(film_id)
INNER JOIN category USING(category_id)
GROUP BY
    film.title,
    category.name,
    category_id
ORDER BY maximum_duration DESC
"""
show_query(base_query)

+-----------+----------------+
|       name|maximum_duration|
+-----------+----------------+
|      Games|             185|
|      Music|             185|
|     Sci-Fi|             185|
|     Comedy|             185|
|     Action|             185|
|     Travel|             185|
|     Action|             185|
|  Animation|             185|
|     Travel|             185|
|  Animation|             185|
|      Games|             184|
|     Sports|             184|
|  Animation|             184|
|  Animation|             184|
|     Family|             184|
|    Foreign|             184|
|    Foreign|             184|
|   Classics|             184|
|        New|             183|
|Documentary|             183|
+-----------+----------------+
only showing top 20 rows



# Using the essential as a subquery

In [103]:
query = """SELECT title
        FROM film
        WHERE length> ANY(
        (
        SELECT MAX( length )
        FROM film
        INNER JOIN film_category USING(film_id)
        GROUP BY  category_id)
        )"""
show_query(query)

AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "any(scalarsubquery())" due to data type mismatch: Parameter 1 requires the "BOOLEAN" type, however "scalarsubquery()" has the type "SMALLINT".; line 3 pos 22;
'Project ['title]
+- 'Filter (length#262 > any(scalar-subquery#1050 []))
   :  +- Aggregate [category_id#188], [max(length#1060) AS max(length)#1052]
   :     +- Project [film_id#1053, title#1054, description#1055, release_year#1056, language_id#1057, rental_duration#1058, rental_rate#1059, length#1060, replacement_cost#1061, rating#1062, last_update#1063, special_features#1064, fulltext#1065, category_id#188, last_update#189]
   :        +- Join Inner, (film_id#1053 = cast(film_id#187 as int))
   :           :- SubqueryAlias film
   :           :  +- View (`film`, [film_id#1053,title#1054,description#1055,release_year#1056,language_id#1057,rental_duration#1058,rental_rate#1059,length#1060,replacement_cost#1061,rating#1062,last_update#1063,special_featur

In [76]:
query = """
SELECT title
FROM film
WHERE length >= ANY(
        (
        SELECT MAX( length )
        FROM film
        INNER JOIN film_category USING(film_id)
        GROUP BY  category_id 
        )
    );

"""
show_query(query)

AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "any(scalarsubquery())" due to data type mismatch: Parameter 1 requires the "BOOLEAN" type, however "scalarsubquery()" has the type "SMALLINT".; line 4 pos 16;
'Project ['title]
+- 'Filter (length#262 >= any(scalar-subquery#757 []))
   :  +- Aggregate [category_id#188], [max(length#767) AS max(length)#759]
   :     +- Project [film_id#760, title#761, description#762, release_year#763, language_id#764, rental_duration#765, rental_rate#766, length#767, replacement_cost#768, rating#769, last_update#770, special_features#771, fulltext#772, category_id#188, last_update#189]
   :        +- Join Inner, (film_id#760 = cast(film_id#187 as int))
   :           :- SubqueryAlias film
   :           :  +- View (`film`, [film_id#760,title#761,description#762,release_year#763,language_id#764,rental_duration#765,rental_rate#766,length#767,replacement_cost#768,rating#769,last_update#770,special_features#771,fulltext#772])
   : 

## Other example

In [64]:
query = """
SELECT
    title,
    category_id
FROM
    film
INNER JOIN film_category
        USING(film_id)
WHERE
    category_id = ANY((
        SELECT
            category_id
        FROM
            category
        WHERE
            NAME = 'Action'
            OR NAME = 'Drama'
    ));

"""
show_query(query)

AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "any(scalarsubquery())" due to data type mismatch: Parameter 1 requires the "BOOLEAN" type, however "scalarsubquery()" has the type "INT".; line 10 pos 18;
'Project ['title, 'category_id]
+- 'Filter (category_id#188 = any(scalar-subquery#620 []))
   :  +- Project [category_id#141]
   :     +- Filter ((NAME#142 = Action) OR (NAME#142 = Drama))
   :        +- SubqueryAlias category
   :           +- View (`category`, [category_id#141,name#142,last_update#143])
   :              +- Relation [category_id#141,name#142,last_update#143] JDBCRelation(category) [numPartitions=1]
   +- Project [film_id#255, title#256, description#257, release_year#258, language_id#259, rental_duration#260, rental_rate#261, length#262, replacement_cost#263, rating#264, last_update#265, special_features#266, fulltext#267, category_id#188, last_update#189]
      +- Join Inner, (film_id#255 = cast(film_id#187 as int))
         :- SubqueryAli

# ALL

## Spark documentation

In [88]:
query = "SELECT ANY(col) FROM VALUES (true), (true), (true) AS tab(col);"
show_query(query)

+--------+
|any(col)|
+--------+
|    true|
+--------+



## ALL works differently on Spark

In [95]:
query = """
SELECT
    title,
    category_id
FROM
    film
INNER JOIN film_category
        USING(film_id)
WHERE
    category_id = ANY(
        (
        SELECT
            category_id
        FROM
            category
        WHERE
            NAME = 'Action'
            OR NAME = 'Drama'
        )
    );

"""
show_query(query)

AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "any(scalarsubquery())" due to data type mismatch: Parameter 1 requires the "BOOLEAN" type, however "scalarsubquery()" has the type "INT".; line 10 pos 18;
'Project ['title, 'category_id]
+- 'Filter (category_id#188 = any(scalar-subquery#976 []))
   :  +- Project [category_id#141]
   :     +- Filter ((NAME#142 = Action) OR (NAME#142 = Drama))
   :        +- SubqueryAlias category
   :           +- View (`category`, [category_id#141,name#142,last_update#143])
   :              +- Relation [category_id#141,name#142,last_update#143] JDBCRelation(category) [numPartitions=1]
   +- Project [film_id#255, title#256, description#257, release_year#258, language_id#259, rental_duration#260, rental_rate#261, length#262, replacement_cost#263, rating#264, last_update#265, special_features#266, fulltext#267, category_id#188, last_update#189]
      +- Join Inner, (film_id#255 = cast(film_id#187 as int))
         :- SubqueryAli

# EXISTS

In [26]:
query = """
SELECT first_name,
       last_name
FROM customer c
WHERE EXISTS
    (SELECT 1
     FROM payment p
     WHERE p.customer_id = c.customer_id
       AND amount > 11 )
ORDER BY first_name,
         last_name;

"""
show_query(query)

+----------+---------+
|first_name|last_name|
+----------+---------+
|     Karen|  Jackson|
|      Kent|Arsenault|
|  Nicholas| Barfield|
|  Rosemary|  Schmidt|
|     Tanya|  Gilbert|
|  Terrance|    Roush|
|   Vanessa|     Sims|
|  Victoria|   Gibson|
+----------+---------+



# NOT EXISTS

In [27]:
query = """
SELECT first_name,
       last_name
FROM customer c
WHERE NOT EXISTS
    (SELECT 1
     FROM payment p
     WHERE p.customer_id = c.customer_id
       AND amount > 11 )
ORDER BY first_name,
         last_name;
"""
show_query(query)

+----------+-----------+
|first_name|  last_name|
+----------+-----------+
|     Aaron|      Selby|
|      Adam|      Gooch|
|    Adrian|      Clary|
|     Agnes|     Bishop|
|      Alan|       Kahn|
|    Albert|     Crouse|
|   Alberto|    Henning|
|      Alex|    Gresham|
| Alexander|    Fennell|
|    Alfred|   Casillas|
|   Alfredo|    Mcadams|
|     Alice|    Stewart|
|    Alicia|      Mills|
|     Allan|    Cornish|
|     Allen|Butterfield|
|   Allison|    Stanley|
|      Alma|     Austin|
|     Alvin|    Deloach|
|    Amanda|     Carter|
|     Amber|      Dixon|
+----------+-----------+
only showing top 20 rows



# Double checking the errors with pure PostgresSQL

## Installing psycopg2-binary

In [35]:
#!pip install psycopg2-binary

  Obtaining dependency information for psycopg2-binary from https://files.pythonhosted.org/packages/ce/85/62825cabc6aad53104b7b6d12eb2ad74737d268630032d07b74d4444cb72/psycopg2_binary-2.9.9-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 8.9 MB/s eta 0:00:0000:0100:01


In [36]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from sqlalchemy.orm import Session

In [47]:
def show_postgres_query(query,con):
    return pd.read_sql_query(query,con = con)

In [38]:
url = URL.create(
                drivername="postgresql+psycopg2",
                username="postgres",
                password="1234",
                host="localhost",
                port="5432",
                database="dvdrental",
            )

In [40]:
db_engine = create_engine(url=url)

## ANY examples

In [106]:
query = """SELECT title
FROM film
WHERE length >= ANY(
    SELECT MAX( length )
    FROM film
    INNER JOIN film_category USING(film_id)
    GROUP BY  category_id );"""
show_postgres_query(query,con = db_engine).head()

,title
0,Alley Evolution
1,Analyze Hoosiers
2,Anonymous Human
3,Baked Cleopatra
4,Casualties Encino


In [104]:
query = """
SELECT
    title,
    category_id
FROM
    film
INNER JOIN film_category
        USING(film_id)
WHERE
    category_id = ANY(
       (
        SELECT
            category_id
        FROM
            category
        WHERE
            NAME = 'Action'
            OR NAME = 'Drama')
    );

"""
show_postgres_query(query,con = db_engine).head()

,title,category_id
0,Amadeus Holy,1
1,American Circus,1
2,Antitrust Tomatoes,1
3,Apollo Teen,7
4,Ark Ridgemont,1


## ALL example

In [52]:
query = """
SELECT
    film_id,
    title,
    length
FROM
    film
WHERE
    length > ALL(SELECT ROUND(AVG (length),2) FROM film GROUP BY rating)
ORDER BY
    length;

"""
show_postgres_query(query,con = db_engine).head()

,film_id,title,length
0,207,Dangerous Uptown,121
1,86,Boogie Amelie,121
2,403,Harry Idaho,121
3,93,Brannigan Sunrise,121
4,704,Pure Runner,121


# Exercise: redo all the above using Pyspark API and investigate the ANY and ALL Spark SQL cases

## Tip 1: https://www.w3schools.com/sql/sql_any_all.asp

## Tip 2: https://spark.apache.org/docs/latest/api/sql/index.html#any